### Set standard imports

In [54]:
import matplotlib,lmfit
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib tk
#for more comfort use
#%matplotlib qt
import KiMoPack.plot_func as pf
from importlib import reload
reload(pf)

Qt was found consider switching to qt mode with %matplotlib qt (more comfortable)
Plot_func version 7.5.5
was imported from path:
 C:\Users\Jens\anaconda3\Lib\site-packages\KiMoPack
The current working folder is:
 D:\Dropbox\coding\github\KiMoPack\Workflow_tools


<module 'KiMoPack.plot_func' from 'C:\\Users\\Jens\\anaconda3\\Lib\\site-packages\\KiMoPack\\plot_func.py'>

### Load and inspect Data 

In [132]:
ta=pf.TA('XES_on.SIA',path='Data',sep=',',units='eV',sort_indexes=True,decimal='.',data_type='X-ray emission intensity',baseunit='ps')   #type filename and path (leave path empty if in the same folder)
#ta=pf.TA('XES_diff.SIA',path='Data',sep=',',units='eV',sort_indexes=True,decimal='.',data_type='X-ray emission differential')
#ta=pf.TA("sample_1.hdf5",path="Data")  #reload saved projects using the same syntax
#ta=pf.TA('gui')                       #use a GUI to select data

### Shape data and plot Raw images

In [133]:
plt.close('all')
ta.rel_wave=np.arange(7053,7061,1)      # use to manually select interesting wavelength for the kinetics
ta.wavelength_bin=1                                        # width of wavelength bin in kinetics
ta.rel_time=[-0.5,-0.1,0,0.1,0.3,0.5,1,1.5,3,5,10,15,20]      # use to manually set interesting times for the spectra
ta.time_width_percent=0                                    # rebinning of time_points in percent
ta.timelimits=[-1,200]                                     # use to manually limit the range to plot (and fit)     
ta.log_scale=False                                         # use to plot the 2d plots with logarithmic intensity scale
ta.bordercut=[7045,7065]                                      # use to set the outer wavelength limits
#ta.scattercut=[522,605]                                     # this region is blanked out to block scatter, this can be a list of regions
ta.intensity_range=[0,7]                           # set a value 5e-3 or a range [-1e-3,5e-3] for 2d plotting
ta.Plot_RAW()                                      # Plot Raw Spectra

## Apply some gentle smoothing?

In [134]:
if 0:
    ta.ds=pf.Frame_golay(ta.ds,7,3)
    ta.Plot_RAW() 

### Perform fast fitting using independent exponential decay (first order)

In [135]:
import function_library as func
reload(func)

<module 'function_library' from 'D:\\Dropbox\\coding\\github\\KiMoPack\\Workflow_tools\\function_library.py'>

In [136]:
plt.close('all')                                             # Close old spectra
ta.mod=func.P32                                        # Choose the model 'exponential', 'consecutive' and 'fast_consecutive'are build in
par=lmfit.Parameters()                                       # create empty parameter object

par.add('k0',value=1/0.2,vary=True)                                # Add second rate component
par.add('k1',value=1/4.4,vary=True)                                # Add one rate component
par.add('k2',value=1/0.15,vary=True)                                 # Add second rate component
par.add('k3',value=1/16,vary=True)
par.add('t0',value=0,min=-2,max=2,vary=False)                       # Allow the arrival time to adjust (for all)
par.add('resolution',value=0.086,min=0.04,max=0.5,vary=False)       # Allow the instrument response to adjust (for all)
par.add('explicit_GS')
#par.add('infinite')                                                # Keyword for an non decaying component
par.add('background')                                             # Keyword to fit the Background Disable the background substraction in the loading cell for best effect
ta.log_fit=False                                               # fit in linear or log scale
if 0:
    for key in par.keys():
        par[key].vary=False
ta.par=par                                                     # write parameter object into file for fitting
ta.Fit_Global()                                 # trigger fitting
ta.error_matrix_amplification=10;                              # Choose how much stronger should error be plotted                                                    # 2D plots in linear or log scale
ta.Plot_fit_output()                                 # plot the fit output

we use adaptive mode for nelder
415.880350028306
415.9473165867404
415.8120829588317
415.9666355000327
415.9391632257606
415.8094430038218
415.77024000356545
415.74192575803136
415.6914281952164
415.6631062369145
415.61692183319724
415.5782500820171
415.51844218276756
415.453208163779
415.3611352120048
415.34583483800293
415.2614166333843
415.2766018301519
415.0439390779759
414.8938684257722
414.82571410250875
414.63997361406393
414.77045791042065
414.49131934813124
414.31750091777894
414.0195653493941
413.8347314438489
413.77401711247813
413.5862971077486
413.9360332672545
414.93076137265155
414.27992300781295
413.7287582163038
414.47595664890093
413.9671391149163
414.2156531980321
413.81351462929905
413.67107760165806
413.4495561202403
413.3908980892429
413.4615332673211
413.6930903524741
413.54046446582
413.50827242343024
413.4321812424305
413.89678805182814
413.407914058006
413.40586121914896
413.4562822591081
413.3947957550894
413.6794807399524
413.35563349503343
413.3401850059209

### Calculate the absolute spectra

In [137]:
for col in ta.re['DAC'].columns:
    ta.re['DAC'].loc[:,col]+=ta.re['DAC'].background
ta.re['DAC']=ta.re['DAC'].drop('background',axis=1)
ta.Plot_fit_output(0)

In [ ]:
ta.Save_Powerpoint()